In [ ]:
import tweepy
import re
from config import BEARER_TOKEN, API_KEY, API_SECRET_KEY, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

# Function to authenticate with Twitter API
def authenticate_twitter(api_key, api_secret_key, access_token, access_token_secret):
    auth = tweepy.OAuthHandler(api_key, api_secret_key)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    return api

# Function to find emails in text using regex
def find_emails(text):
    email_pattern = r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+'
    emails = re.findall(email_pattern, text)
    return emails

# Function to fetch replies from a specific tweet and extract emails
def fetch_replies(api, tweet_id, username):
    replies = []
    for tweet in tweepy.Cursor(api.search_tweets, q='to:{}'.format(username), since_id=tweet_id, tweet_mode='extended').items():
        if hasattr(tweet, 'in_reply_to_status_id_str'):
            if (tweet.in_reply_to_status_id_str == tweet_id):
                replies.append(tweet.full_text)
    return replies

# Main function
def main():
    bearer_token = BEARER_TOKEN
    api_key = API_KEY
    api_secret_key = API_SECRET_KEY
    access_token = ACCESS_TOKEN
    access_token_secret = ACCESS_TOKEN_SECRET

    # Authenticate and create API object
    api = authenticate_twitter(api_key, api_secret_key, access_token, access_token_secret)

    # Tweet ID and username of the original tweet
    tweet_id = 'TWEET_ID'  # Example: '1426189478389237763'
    username = 'USERNAME'  # Example: 'elonmusk'

    # Fetch replies
    replies = fetch_replies(api, tweet_id, username)

    # Extract emails from replies
    all_emails = []
    for reply in replies:
        emails = find_emails(reply)
        all_emails.extend(emails)

    # Print extracted emails
    print("Extracted Emails:")
    for email in set(all_emails):
        print(email)

if __name__ == "__main__":
    main()


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re
import time

# Path to the ChromeDriver executable
driver_path = 'D:\\hp5cd\\New folder\\Web_Scraping\\chromedriver-win64\\chromedriver.exe'

# Create a Service object with the path to the ChromeDriver
service = Service(driver_path)

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(service=service)

# Navigate to the Twitter replies section
twitter_url = "https://x.com/amznsellerhelp/with_replies"
driver.get(twitter_url)
time.sleep(60)

# Scroll and load more replies
SCROLL_PAUSE_TIME = 2
emails = set()
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for new tweets to load
    time.sleep(SCROLL_PAUSE_TIME)

    # Get the current scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

    # Fetch tweets/replies
    tweets = driver.find_elements(By.XPATH, "//div[@data-testid='tweetText']")
    for tweet in tweets:
        text = tweet.text
        found_emails = re.findall(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', text)
        emails.update(found_emails)

# Print all extracted emails
print("Extracted Emails:")
for email in emails:
    print(email)

# Close the browser
driver.quit()


Extracted Emails:


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re
import time

# Path to the ChromeDriver executable
driver_path = 'D:\\hp5cd\\New folder\\Web_Scraping\\chromedriver-win64\\chromedriver.exe'

# Create a Service object with the path to the ChromeDriver
service = Service(driver_path)

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(service=service)

# Navigate to the Twitter replies section
twitter_url = "https://x.com/Sell_on_Amazon/with_replies"
driver.get(twitter_url)
time.sleep(60)  # Wait for page to load

# Scroll and load more replies
SCROLL_PAUSE_TIME = 10
last_height = driver.execute_script("return document.body.scrollHeight")
usernames = set()

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Get the current scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

    # Fetch usernames of users who commented
    tweets = driver.find_elements(By.XPATH, "//div[@data-testid='User-Names']//span[contains(text(),'@')]")
    for tweet in tweets:
        username = tweet.text.strip()
        if username.startswith('@'):
            usernames.add(username)

# Visit each user's profile to find email in bio
emails = set()
for username in usernames:
    profile_url = f"https://x.com/{username[1:]}"
    driver.get(profile_url)
    time.sleep(3)  # Wait for profile to load
    
    try:
        bio = driver.find_element(By.XPATH, "//div[@data-testid='UserDescription']").text
        print(f"Bio for {username}: {bio}")  # Debugging: Print the bio text
        found_emails = re.findall(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', bio)
        if found_emails:
            print(f"Found emails for {username}: {found_emails}")
        emails.update(found_emails)
        
        # Check if there's a website link
        try:
            website = driver.find_element(By.XPATH, "//div[@data-testid='UserProfileHeader_Items']//a").get_attribute('href')
            print(f"Visit the user's website for more info: {website}")
        except:
            pass
        
    except Exception as e:
        print(f"Error visiting profile {username}: {e}")
        continue

# Print all extracted emails
print("Extracted Emails:")
for email in emails:
    print(email)

# Close the browser
driver.quit()


Extracted Emails:


In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re
import time

# Path to the ChromeDriver executable
driver_path = 'D:\\hp5cd\\New folder\\Web_Scraping\\chromedriver-win64\\chromedriver.exe'

# Create a Service object with the path to the ChromeDriver
service = Service(driver_path)

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(service=service)

# Navigate to the Twitter replies section
twitter_url = "https://x.com/Sell_on_Amazon/with_replies"
driver.get(twitter_url)
time.sleep(30)  # Wait for initial page load

# Scroll and load more replies
SCROLL_PAUSE_TIME = 10
MAX_SCROLL_ATTEMPTS = 10  # Max scroll attempts to ensure all content is loaded
last_height = driver.execute_script("return document.body.scrollHeight")
usernames = set()

scroll_attempts = 0
while scroll_attempts < MAX_SCROLL_ATTEMPTS:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Get the current scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("Reached the bottom of the page or no more content to load.")
        break
    last_height = new_height
    scroll_attempts += 1

    # Debugging: Print scroll attempt
    print(f"Scroll attempt {scroll_attempts}...")

    # Fetch usernames of users who commented
    tweets = driver.find_elements(By.XPATH, "//a[@role='link' and contains(@href, '/')]")
    for tweet in tweets:
        username = tweet.get_attribute("href").split("/")[-1]
        print(f"Found username: {username}")  # Debugging: Print each found username
        if username.startswith('@'):
            usernames.add(username)

# Debugging: Print the total number of usernames found
print(f"Total usernames found: {len(usernames)}")

# Visit each user's profile to find email in bio
emails = set()
for username in usernames:
    profile_url = f"https://x.com/{username}"
    print(f"Visiting profile: {profile_url}")  # Debugging: Print each profile being visited
    driver.get(profile_url)
    time.sleep(5)  # Wait for profile to load
    
    try:
        bio = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[@data-testid='UserDescription']"))
        ).text
        print(f"Bio for {username}: {bio}")  # Debugging: Print the bio text
        found_emails = re.findall(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', bio)
        if found_emails:
            print(f"Found emails for {username}: {found_emails}")
        emails.update(found_emails)
        
        # Check if there's a website link
        try:
            website = driver.find_element(By.XPATH, "//div[@data-testid='UserProfileHeader_Items']//a").get_attribute('href')
            print(f"Visit the user's website for more info: {website}")
        except Exception as e:
            print(f"No website link found for {username}: {e}")
        
    except Exception as e:
        print(f"Error visiting profile {username}: {e}")
        continue

# Print all extracted emails
print("Extracted Emails:")
for email in emails:
    print(email)

# Close the browser
driver.quit()


Scroll attempt 1...
Found username: keyboard_shortcuts
Found username: home
Found username: home
Found username: explore
Found username: notifications
Found username: messages
Found username: grok
Found username: communities
Found username: premium_sign_up
Found username: KushBhakkad
Found username: post
Found username: header_photo
Found username: photo
Found username: amznsellerhelp
Found username: following
Found username: verified_followers
Found username: JenC_FBA
Found username: JenC_FBA
Found username: JenC_FBA
Found username: 1820114365690896511
Found username: amazonfba?src=hashtag_click
Found username: amazonfbm?src=hashtag_click
Found username: 1
Found username: analytics
Found username: Sell_on_Amazon
Found username: Sell_on_Amazon
Found username: Sell_on_Amazon
Found username: 1820465236387778633
Found username: JenC_FBA
Found username: analytics
Found username: ecom_EU
Found username: ecom_EU
Found username: ecom_EU
Found username: 1820440343264817249
Found username: 1
Fo

In [ ]:
import tweepy
import re
import time


def twitte_auth():
    try:
        consumer_key = API_KEY
        consumer_secret = API_SECRET_KEY
        access_token = ACCESS_TOKEN
        access_secret = ACCESS_TOKEN_SECRET

        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)
    except:
        print('Cant login tweeter.')
    return auth


def get_client():
    auth = twitte_auth()
    client = tweepy.API(auth, wait_on_rate_limit=True)
    return client



client = get_client()
print('Logged In.')

KEY = str(input('[?] Put your email keyword:'))
DELAY = int(input('[?] How many time delay between each scrape:'))

textfile = open("emails.txt", "w")

for status in tweepy.Cursor(client.search, q=KEY, since="2021-05-01").items():
    try:
        match = re.search(r'[\w\.-]+@[\w\.-]+', status.text)
        email = match.group(0)
        print(email.lower())
    except:
        continue

    try:
        textfile.write(email.lower() + "\n")
    except:
        continue
    time.sleep(DELAY)